In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

In [73]:
# 파일 읽기
sample =pd.read_csv("sample_submission.csv",encoding='cp949') #제출용
profile = pd.read_csv("train_profiles.csv",encoding= 'cp949') #train 나이
test_click = pd.read_csv("test_clickstreams.tab",encoding='cp949',sep = '\t') 
test_search = pd.read_csv("test_searchkeywords.tab",encoding='cp949',sep = '\t')
train_click = pd.read_csv("train_clickstreams.tab",encoding='cp949',sep = '\t')
train_search = pd.read_csv("train_searchkeywords.tab",encoding='cp949',sep = '\t')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
# profile GROUP 변수 분리
# **************한 번만 실행해야해요*********************
profile['gender'] = profile.GROUP.apply(lambda x: x[0]) # 성별
profile['age'] = profile.GROUP.apply(lambda x: x[1:]) # 나이
del profile['GROUP'] # GROUP 제거

In [75]:
train_click = pd.merge(train_click,profile) # train_click + profile
train_search = pd.merge(train_search,profile) # train_search + profile
click = pd.merge(train_click,test_click)  # clicks 통합
search = pd.merge(train_search, test_search) # search 통합

In [77]:
pd.pivot_table(train_click, values='SITE_CNT', index='ACT_NM', columns=['gender','age'], 
               aggfunc=len, fill_value=0) 

gender             F                 M              
age              20-    30   40+   20-     30    40+
ACT_NM                                              
A/S센터              6    51    34    16    196    134
B2B중개/보증           1    30     4     1      4     29
CDN솔루션구축/서비스       0     1     0     0      2      0
CI/로고 디자인          5     3     4    19     75     15
CRM/영업관리          38    16     9   102      7     66
DB관련개발             1     3     1     0      3     15
DEET/MEET학원      129    22     3    11     11    237
DVD판매/렌탈           2     4     1     2     18    130
ERP/SCM            8    16     1    29     16     75
ERP/그룹웨어개발        81   212    32   171    345    149
FTP                3     7     4     4      5      4
Graphic관련개발      108   413   111    81    201    379
IDC               29    11    29    82    156    188
ISP               12    32    47    43    178    118
IT/정보통신전문지       103    79    62    47     92    192
IT뉴스            1953  2773  3491  3193  13788  20697
IT전문교육            72    52   104    35    125    117
LEET학원             9     5     1     3      3     55
LOG/CRM솔루션         6     1     2     7      8     22
MMORPG          2301  2430  1689  3906   8814   5938
MP3브랜드             9     5     3    16     61     87
MP3쇼핑몰             2     3     4     5     20     36
MRO B2B           32   125    52    69    119    242
NGC                0     0     0     0      3      0
OS관련개발           180   165   531   154    373    434
PC방 정보             2     1    11     3    215     22
PC방프랜차이즈          17     2     2     2     10      9
PMP/내비게이션 커뮤니티     0     4     0    11     40    160
PSP                1     0     0     1     12      2
RSS리더              0    59    12     2     49    123
...              ...   ...   ...   ...    ...    ...
헬스보충제쇼핑몰          19   104    84   239    158     19
협동/공제조합            0     1    16     2      9     61
호두과자브랜드            0     2     0     1      0      4
호신용품쇼핑몰            0     0     0     1      2      3
호주/뉴질랜드            0     1     1     0      0      2
호텔               336   197    85   188    214    153
호프전문점              0     1     0     4      1     10
홈스테이               5     5     7     1     12     12
홈인테리어용품쇼핑몰       373   928   171   122    940    338
홈페이지계정             4    10    19     5     38     47
홈페이지제작            42    71    24    38    417     86
화공약품               0     1     0     0      2      0
화장/메이크업서비스        11     1     0     0      0      0
화장지제조              0   124     0     1      3      4
화장품브랜드쇼핑몰       1582  1003   517   404   1057    759
화장품브랜드업체        1601  1056   209   297    681    452
화장품샘플쇼핑몰           3     9     1     0      5      2
화장품제조            484   423   121   149    456    207
화학 무역업체            0     0     1     1      2     13
화학B2B              0     2     0     0      0      0
환경기관/단체            3     5    10     2     12     17
환경뉴스              50    33    37    28     87    140
환경운동/캠페인           7    12    43    10      7     66
휠타이어쇼핑몰            7     2    21     2     42     26
휴대폰 커뮤니티          34    38    72    71    351    257
휴대폰/모바일단말기        11     9    10     4     28     38
휴대폰결제              4    16    12     3     38     28
휴대폰브랜드           124   159   115   100    417    518
휴대폰쇼핑몰           576   212   216   169   1194   1061
휴대폰악세사리쇼핑몰       403   209   109   184    681    532

[1216 rows x 6 columns]

In [47]:
del train_click['SITE'] #SITE 변수 삭제

In [169]:
train_click = train_click.dropna()
x = train_click[['SITE_CNT','ST_TIME']]
y = train_click['age']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.24, random_state=0)


In [173]:
print(len(X_train) == len(y_train)
,len(X_test) == len(y_test))

True True


In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1.0, random_state=0)
# 로지스틱 회귀에서의 C와 반대의 개념. 모델을 조율해주는 값이라고 보면 됨.
svm.fit(X_train, y_train)
y_pred_svc = svm.predict(X_test)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_svc))


np.random.seed(0)
X_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(X_xor[:, 0] > 0,
                       X_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)

plt.scatter(X_xor[y_xor == 1, 0],
            X_xor[y_xor == 1, 1],
            c='b', marker='x',
            label='1')
plt.scatter(X_xor[y_xor == -1, 0],
            X_xor[y_xor == -1, 1],
            c='r',
            marker='s',
            label='-1')

plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc='best')
plt.tight_layout()
# plt.savefig('./figures/xor.png', dpi=300)
plt.show()


svm = SVC(kernel='rbf', C=10.0, random_state=0, gamma=0.10)
svm.fit(X_xor, y_xor)
y_pred_ksvc = svm.predict(X_xor)
print('Accuracy: %.2f' % accuracy_score(y_xor, y_pred_ksvc))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
